In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib, json, os, sys
from lime.lime_tabular import LimeTabularExplainer
from sklearn import set_config
set_config(transform_output='pandas')

In [2]:
import json
with open ('config/filepaths.json') as f:
    FPATHS = json.load(f)
    
FPATHS.keys()

dict_keys(['data', 'models', 'images'])

In [20]:
FPATHS

{'data': {'raw': {'full': 'data/ames-housing-dojo-for-ml.csv',
   'eda': 'data/ames-housing-dojo-for-ml-eda.csv'},
  'ml': {'train': 'data/training-data.joblib',
   'test': 'data/testing-data.joblib'}},
 'models': {'linear_regression': 'models/linear_regression/linreg.joblib',
  'random_forest': 'models/random_forest/rf_reg.joblib'},
 'images': {'banner': 'images/app-banner.png'}}

In [3]:
def load_data(fpath):
    df = pd.read_csv(fpath)
    df = df.set_index("PID")
    return df

In [4]:
# Load dataset
df = load_data(fpath = FPATHS['data']['raw']['full'])
df.head()

,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Utilities,Neighborhood,Bldg Type,House Style,Overall Qual,...,Garage Area,Garage Qual,Garage Cond,Paved Drive,Fence,SalePrice,Month,Year,Total Full Baths,Total Half Baths
PID,,,,,,,,,,,,,,,,,,,,,
907227090,RL,60.0,7200,Pave,NaN,AllPub,CollgCr,1Fam,1Story,5,...,297.0,TA,TA,Y,MnPrv,119900.0,3,2006,1.0,0.0
527108010,RL,134.0,19378,Pave,NaN,AllPub,Gilbert,1Fam,2Story,7,...,576.0,TA,TA,Y,NaN,320000.0,3,2006,3.0,1.0
534275170,RL,NaN,12772,Pave,NaN,AllPub,NAmes,1Fam,1Story,6,...,301.0,TA,TA,Y,NaN,151500.0,4,2007,1.0,0.0
528104050,RL,114.0,14803,Pave,NaN,AllPub,NridgHt,1Fam,1Story,10,...,1220.0,TA,TA,Y,NaN,385000.0,6,2008,3.0,0.0
533206070,FV,32.0,3784,Pave,Pave,AllPub,Somerst,TwnhsE,1Story,8,...,476.0,TA,TA,Y,NaN,193800.0,2,2007,3.0,0.0


In [5]:
def load_Xy_data(fpath):
    return joblib.load(fpath)

In [6]:
# Call function for train data
X_train, y_train = load_Xy_data(fpath=FPATHS['data']['ml']['train'])
# Call function for test data
X_test, y_test = load_Xy_data(fpath=FPATHS['data']['ml']['test'])

In [7]:
# Define function to load in model from dictionary with joblib
def load_model_ml(fpath):
    return joblib.load(fpath)

In [21]:
rf = load_model_ml(fpath = FPATHS['models']['random_forest'])
rf

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Living Area Sqft',
                                                   'Lot Frontage', 'Bedroom',
                                                   'Total Full Baths']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Bldg Type', 'MS Zoning',
                                                   'Street', 'Alley',
                                                   'Utilities'])],
                                   verbose_feature_names_out=False)),
                ('reg', LinearRegression())])

In [12]:
# Get list of all the features
X_train.columns

Index(['Living Area Sqft', 'Lot Frontage', 'Bldg Type', 'Bedroom',
       'Total Full Baths', 'MS Zoning', 'Street', 'Alley', 'Utilities'],
      dtype='object')